`data_preprocessed_cut_2_Valence.csv`と`data_preprocessed_cut_2_Arousal.csv`のmodelをimportしてpredをcsv出力する
相関係数rを出す

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import numpy as np
import pandas as pd

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

import json

/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


In [2]:
import gensim
gensim_model = gensim.models.KeyedVectors.load_word2vec_format('./word2vec/data.bin', binary=True)

In [3]:
weights = gensim_model.vectors

In [4]:
embedding_dim = weights.shape[1]
print(embedding_dim,weights.shape[0])

300 3000000


In [5]:
weights = np.append(weights,np.zeros((1,embedding_dim)),axis=0)
# 末尾にunknown_wordを追加

In [6]:
vocab_size = weights.shape[0]
print(vocab_size)

3000001


In [7]:
out_size = 1

In [8]:
cuda = torch.cuda.is_available()

In [9]:
# wordのindexを取得
# print(gensim_model.wv.vocab['always'].index)
# 100番目のwordを取得
# print(gensim_model.wv.index2word[100])

In [10]:
import re
import nltk

In [11]:
def prepare_sequence(seq):
    vocab = gensim_model.wv.vocab
    idxs = [vocab[w].index if w in vocab else vocab_size - 1 for w in seq]
    res = torch.tensor(idxs, dtype=torch.long)
    if cuda:
        res = res.cuda()
    return res

In [12]:
def sentence2vec(sentence):
    w_list = sentence.split()
    res_seq = prepare_sequence(w_list)
    return res_seq

In [13]:
# s = "I'm always fucking you."
# sentence2vec(s)

In [14]:
class LSTMTagger(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)

        # The linear layer that maps from hidden state space to tag space
        self.out = nn.Linear(hidden_dim, tagset_size)
        self.hidden = self.init_hidden(1)

    def init_hidden(self,batch_size):
        # Before we've done anything, we dont have any hidden state.
        # Refer to the Pytorch documentation to see exactly
        # why they have this dimensionality.
        # The axes semantics are (num_layers, minibatch_size, hidden_dim)
        h = torch.zeros(1, batch_size, self.hidden_dim)
        c = torch.zeros(1, batch_size, self.hidden_dim)
        if cuda:
            h = h.cuda()
            c = c.cuda()
        return (h,c)

    def forward(self, sentence, lengths):
        embeds = self.word_embeddings(sentence)
#         print(embeds.size())
#         batch_size = embeds.size()[0]
        packed = nn.utils.rnn.pack_padded_sequence(embeds,lengths,batch_first=True)
        lstm_output, self.hidden = self.lstm(packed, self.hidden)
        unpacked,_ = nn.utils.rnn.pad_packed_sequence(lstm_output,batch_first=True)
        # batch * hidden 
        output = self.out(unpacked[:,-1,:])
        output = F.tanh(output)
        return output


In [15]:
# model.state_dict().keys()

In [16]:
def make_model(hidden_dim):
    # 学習済みパラメータ
    torch.manual_seed(1)
    model = LSTMTagger(embedding_dim, hidden_dim, vocab_size, out_size)
    model.word_embeddings = nn.Embedding.from_pretrained(torch.from_numpy(weights).float())
    return model

In [17]:
def save_model(model,model_name):
    model_state_dict = model.state_dict()
    model_state_dict.pop('word_embeddings.weight')
    torch.save(model_state_dict,model_name)

In [18]:
def load_model(hidden_dim,model_name):
    torch.manual_seed(1)
    model = LSTMTagger(embedding_dim, hidden_dim, vocab_size, out_size)
    model_state_dict = torch.load(model_name)
    model_state_dict['word_embeddings.weight'] = torch.from_numpy(weights).float()
    model.load_state_dict(model_state_dict)
    # Freeze
    model.word_embeddings.weight.requires_grad = False
    return model

In [19]:
# vad_type='Valence'
# data_cut = pd.read_csv('./data_preprocessed_{0}.csv'.format(vad_type),encoding='utf-16')
# data_cut = data_cut[data_cut['words']>=2]
# X_train = data_cut[data_cut['data_type']=='train']['reg'].as_matrix()
# Y_train = data_cut[data_cut['data_type']=='train'][['{0}_reg'.format(vad_type)]].as_matrix()
# samples = []
# for sentence,target in zip(X_train,Y_train):
#     sentence_vec = sentence2vec(sentence)
#     y_hat = torch.tensor(target, dtype=torch.float)
#     if cuda:
#         y_hat = y_hat.cuda()
#     samples.append((sentence_vec,y_hat))

In [20]:
def pad_tensor(vec, pad, dim):
    """
    args:
        vec - tensor to pad
        pad - the size to pad to
        dim - dimension to pad

    return:
        a new tensor padded to 'pad' in dimension 'dim'
    """
    pad_size = list(vec.shape)
    pad_size[dim] = pad - vec.size(dim)
    res = torch.cat([vec, (torch.zeros(*pad_size,dtype=torch.long).cuda() \
                           if cuda else torch.zeros(*pad_size,dtype=torch.long))], dim=dim)
    if cuda:
        res = res.cuda()
    return res

class PadCollate:
    """
    a variant of callate_fn that pads according to the longest sequence in
    a batch of sequences
    """

    def __init__(self, dim=0):
        """
        args:
            dim - the dimension to be padded (dimension of time in sequences)
        """
        self.dim = dim

    def pad_collate(self, batch):
        """
        args:
            batch - list of (tensor, label)

        reutrn:
            xs - a tensor of all examples in 'batch' after padding
            ys - a LongTensor of all labels in batch
        """
        lengths = list(map(lambda x: x[0].shape[self.dim], batch))
        # find longest sequence
        max_len = max(lengths)
        # pad according to max_len
        xs = torch.zeros([len(lengths),max_len],dtype=torch.long)
        if cuda:
            xs = xs.cuda()
        for idx,(seq,seqlen) in enumerate(zip(batch,lengths)):
            xs[idx,:seqlen] = seq[0]
        ys = torch.FloatTensor(list(map(lambda x: x[1], batch)))
        lengths_tensor = torch.tensor(lengths)

        if cuda:
            xs = xs.cuda()
            ys = ys.cuda()
            lengths_tensor = lengths_tensor.cuda()
        return xs, ys, lengths_tensor

    def __call__(self, batch):
        return self.pad_collate(batch)

In [21]:
# train_loader = DataLoader(samples,batch_size=4,shuffle=True,collate_fn=PadCollate(dim=0))

In [22]:
# for sam in train_loader:
#     sam
#     print(sam[0].size(),sam[1].size())
#     break

In [23]:
def make_dataset(X,Y):
    ds = []
    for sentence,target in zip(X,Y):
        sentence_vec = sentence2vec(sentence)
        y_hat = torch.tensor(target, dtype=torch.float)
        if cuda:
            y_hat = y_hat.cuda()
        ds.append((sentence_vec,y_hat))
    return ds

In [24]:
def make_pred(hidden_dim,vad_type,model_name,dat_base_name='./data_preprocessed'):
    # args
    # dat_base_name : _{Valence,Arousal}.csv以前へのパス
    # save_dor : modelとjsonをsaveするdir (中にjson,modelディレクトリを含む)

    #     json_name = '{0}/json/loss_{1}_hidden_dim_{2}_batch_{3}_lr_{4}.json'.format(\
    #                    save_dir,vad_type,hidden_dim,batch_size,learning_rate)

    model = load_model(hidden_dim,model_name)

    if cuda:
        model.cuda()

    data_cut = pd.read_csv('{0}_{1}.csv'.format(dat_base_name,vad_type),encoding='utf-16')
    data_cut = data_cut[data_cut['words']>=2]

    X_train = data_cut[data_cut['data_type']=='train']['reg'].as_matrix()
    X_dev   = data_cut[data_cut['data_type']=='dev']['reg'].as_matrix()
    X_test  = data_cut[data_cut['data_type']=='test']['reg'].as_matrix()
    Y_train = data_cut[data_cut['data_type']=='train'][['{0}_reg'.format(vad_type)]].as_matrix()
    Y_dev   = data_cut[data_cut['data_type']=='dev'][['{0}_reg'.format(vad_type)]].as_matrix()
    Y_test  = data_cut[data_cut['data_type']=='test'][['{0}_reg'.format(vad_type)]].as_matrix()

    loader = {}
    
    batch_size = 1
    
    ds_train = make_dataset(X_train,Y_train)
    loader['train'] = DataLoader(ds_train,batch_size=batch_size,shuffle=False,collate_fn=PadCollate(dim=0))

    ds_dev = make_dataset(X_dev,Y_dev)
    loader['dev'] = DataLoader(ds_dev,batch_size=batch_size,shuffle=False,collate_fn=PadCollate(dim=0))
    
    ds_test = make_dataset(X_test,Y_test)
    loader['test'] = DataLoader(ds_test,batch_size=batch_size,shuffle=False,collate_fn=PadCollate(dim=0))
    

    lis = ['train','dev','test']
    
    y_true = {}
    y_pred = {}
    
    for data_type in lis:
        y_true[data_type] = []
        y_pred[data_type] = []
        for x_batch,y_batch,lengths in loader[data_type]:
            model.zero_grad()
            model.hidden = model.init_hidden(len(x_batch))

            # sort
            lengths, perm_idx = lengths.sort(0, descending=True)
            x_batch = x_batch[perm_idx]
            y_batch = y_batch[perm_idx]
            y_true[data_type].append(y_batch.cpu().detach().numpy()[0])

            y_p = model(x_batch,lengths)
            y_p = y_p.view(-1)
            y_pred[data_type].append(y_p.cpu().detach().numpy()[0])
    
    return y_true,y_pred


In [25]:
save_dir='./dat_model_json/dat_word_cut_l1loss'
hidden_dim = 128
vad_type = 'Valence'
batch_size = 4
learning_rate = 0.001
epoch = 50
model_name = '{0}/model/model_{1}_hidden_dim_{2}_batch_{3}_lr_{4}_epoch_{5}'.format(\
                        save_dir,vad_type,hidden_dim,batch_size,learning_rate,epoch)

y_true_v, y_pred_v = make_pred(hidden_dim,vad_type,model_name,dat_base_name='./data_preprocessed')

/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel/__main__.py:2: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  from ipykernel import kernelapp as app


In [34]:
data_types = ['train','dev','test' ]
for data_type in data_types:
    print("{0} : {1}".format(data_type,np.corrcoef(y_true_v[data_type],y_pred_v[data_type])[0,1]))

train : 0.7211203020973606
dev : 0.6813731184931702
test : 0.6714589264296188


In [35]:
save_dir='./dat_model_json/dat_word_cut_l1loss'
hidden_dim = 64
vad_type = 'Arousal'
batch_size = 4
learning_rate = 0.001
epoch = 50
model_name = '{0}/model/model_{1}_hidden_dim_{2}_batch_{3}_lr_{4}_epoch_{5}'.format(\
                        save_dir,vad_type,hidden_dim,batch_size,learning_rate,epoch)

y_true_a, y_pred_a = make_pred(hidden_dim,vad_type,model_name,dat_base_name='./data_preprocessed')

/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel/__main__.py:2: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  from ipykernel import kernelapp as app


In [36]:
data_types = ['train','dev','test' ]
for data_type in data_types:
    print("{0} : {1}".format(data_type,np.corrcoef(y_true_a[data_type],y_pred_a[data_type])[0,1]))

train : 0.4822053015506005
dev : 0.5122123707767103
test : 0.4766804936908686
